In [ ]:
from daphne import daphne
import os

In [ ]:
directory = 'programs/tests/hoppl-deterministic'
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))

In [ ]:
i=3
cwd='/Users/gw/repos/daphne'
exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
exp



In [ ]:
desugared_ast_json_fname = '/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)
with open(desugared_ast_json_fname, 'w') as f:
    json.dump(exp, f)

In [ ]:
%ls /Users/gw/repos/daphne/../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic

In [ ]:
class Env():
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.data = pmap(zip(parms, args))
        self.outer = outer
        if outer is None:
            self.level = 0
        else:
            self.level = outer.level+1
    def __getitem__(self, item):
        return self.data[item]
    def find(self, var):
        "Find the innermost Env where var appears."
        if (var in self.data):
            return self
        else:
            if self.outer is not None:
                return self.outer.find(var)
            else:
                raise RuntimeError('var "{}" not found in outermost scope'.format(var))
    
    def print_env(self, print_lowest=False):
        print_limit = 1 if print_lowest == False else 0
        outer = self
        while outer is not None:
            if outer.level >= print_limit:
                print('Scope on level ', outer.level)
                if 'f' in outer:
                    print('Found f, ')
                    print(outer['f'].body)
                    print(outer['f'].parms)
                    print(outer['f'].env)
                print(outer,'\n')
            outer = outer.outer

In [ ]:
import copy

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        new_env = copy.deepcopy(self.env)
        return eval(self.body, Env(self.parms, args, new_env)) # [0]

In [ ]:
from pyrsistent import pmap

import torch
import torch.distributions as dist

In [ ]:
def push_addr(alpha, value):
    return alpha + value

penv = {
    '+': torch.add,
    '*':torch.mul,
#     'push-address':push_addr
}

In [ ]:
def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env(penv.keys(), penv.values())
    return env

env = standard_env()
env.find('+')
#['+'](torch.tensor(0.),torch.tensor(1.))

In [ ]:
def eval(x,env=standard_env(),sigma=None):

    print('x',x)
    if isinstance(x,str):
        return env.find(x)[x], sigma
    elif not isinstance(x,list):
        return torch.tensor(x),sigma
    
    op, param, *args = x
    
    if False:
        pass
    
    elif op == 'push-address':
        return '', sigma
    elif op == 'fn':
        print('args',args)
#         param, body = args
        body = args[0]
        return Procedure(param, body, env), sigma # has eval in it
        # param ['alpha', 'x']
        # body ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
        # env
    
    else:
        print('in else. x',x)
        proc, _ = eval(op,env,sigma)
        vals = [''] # TODO: address for fn???
        vals.extend([eval(arg, env, sigma)[0] for arg in args])
        print('vals',vals)

        if isinstance(proc, Procedure): # lambdas, not primitives
            print('in Procedure',proc, vals)
            r, _ = proc(*vals)
        else:
            r = proc(*vals[1:]) # primitives
            print('in primitives',proc, vals)
            
        return r, sigma
    

# eval(['*','a0',3,3])  

# why vals only tensor(8.), where did push add go?

In [ ]:
exp

In [ ]:
proc, sigma = eval(exp)
proc('')

In [ ]:
exp_sq_8 = [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   9]

eval(exp_sq_8)#(torch.tensor(8.))

In [ ]:
exp_inner = ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
eval(exp_inner)#(torch.tensor(8.))

In [ ]:
import sys
sys.setrecursionlimit(10000)

# test driven development

## hash-map

In [ ]:
# eval_hoppl(['hash-map','','a',1,'b',2])

In [46]:
# from old_primitives import penv
# from torch import tensor

# vals = ['',tensor(1.),tensor(2.)]
# penv['+'](vals[1:])

tensor(3.)

In [147]:
# fname = 'programs/tests/hoppl-deterministic/test_{}.daphne.json'.format(i)

# with open(fname) as f:
#      exp = json.load(f)
# exp
# # 

## deterministic-hoppl tests

In [207]:
from tests import load_truth, is_tol
import os, json
from daphne import daphne

import importlib
import evaluator
importlib.reload(evaluator)
from evaluator import evaluate, run_deterministic_tests, ast_helper, eval_hoppl

In [208]:
i=7
fname='test_{}.daphne'.format(i)
exp = ast_helper(fname,directory='programs/tests/hoppl-deterministic')
exp

['fn',
 ['alpha'],
 [['fn',
   ['alpha', 'm'],
   [['m', ['push-address', 'alpha', 'addr2'], 1],
    ['push-address', 'alpha', 'addr1'],
    2]],
  ['push-address', 'alpha', 'addr0'],
  ['fn',
   ['alpha', 'x'],
   ['if',
    ['>', ['push-address', 'alpha', 'addr3'], 0, 'x'],
    ['fn', ['alpha', 'y'], ['+', ['push-address', 'alpha', 'addr4'], 1, 'y']],
    ['fn',
     ['alpha', 'z'],
     ['+', ['push-address', 'alpha', 'addr5'], 2, 'z']]]]]]

In [209]:
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/')
truth = load_truth('programs/tests/hoppl-deterministic/test_{}.truth'.format(i))
truth

4

In [210]:
ret = evaluate(exp, do_log=True)
assert(is_tol(ret, truth))


x ['fn', ['alpha', 'z'], ['+', ['push-address', 'alpha', 'addr5'], 2, 'z']]
args [['+', ['push-address', 'alpha', 'addr5'], 2, 'z']]
return tensor(4)


## all tests

In [181]:
%cd /Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
run_deterministic_tests()

/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5
x ['fn', ['alpha'], ['+', ['push-address', 'alpha', 'addr0'], 5, 2]]
case fn: args [['+', ['push-address', 'alpha', 'addr0'], 5, 2]]
x ['+', ['push-address', 'alpha', 'addr0'], 5, 2]
case else: x ['+', ['push-address', 'alpha', 'addr0'], 5, 2]
x +
x 5
x 2
vals ['', tensor(5), tensor(2)]
case primitives: <function add_primitive at 0x1327901f0> ['', tensor(5), tensor(2)]
FOPPL test 1 passed
x ['fn', ['alpha'], ['sqrt', ['push-address', 'alpha', 'addr0'], 2]]
case fn: args [['sqrt', ['push-address', 'alpha', 'addr0'], 2]]
x ['sqrt', ['push-address', 'alpha', 'addr0'], 2]
case else: x ['sqrt', ['push-address', 'alpha', 'addr0'], 2]
x sqrt
x 2
vals ['', tensor(2)]
case primitives: <function sqrt_primitive at 0x1335365e0> ['', tensor(2)]
FOPPL test 2 passed
x ['fn', ['alpha'], ['*', ['push-address', 'alpha', 'addr0'], 3.0, 8.0]]
case fn: args [['*', ['push-address', 'alpha', 'addr0'], 3.0, 8.0]]
x ['*', ['push-address', 'alpha', 'addr0'], 3.0, 8.0]


RuntimeError: var ""a"" not found in outermost scope